In [ ]:
!pip install scipy.misc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement scipy.misc (from versions: none)
ERROR: No matching distribution found for scipy.misc


In [ ]:



import math

In [ ]:
import pickle
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from matplotlib.pyplot  import imread
import numpy as np
from argparse import ArgumentParser

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
def load_graph(frozen_graph_filename):
    with tf.io.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph


def encoder(loadmodel, input_path, refer_path, outputfolder):
    graph = load_graph(loadmodel)
    prefix = 'import/build_towers/tower_0/train_net_inference_one_pass/train_net/'

    Res = graph.get_tensor_by_name(prefix + 'Residual_Feature:0')
    inputImage = graph.get_tensor_by_name('import/input_image:0')
    previousImage = graph.get_tensor_by_name('import/input_image_ref:0')
    Res_prior = graph.get_tensor_by_name(prefix + 'Residual_Prior_Feature:0')
    motion = graph.get_tensor_by_name(prefix + 'Motion_Feature:0')
    bpp = graph.get_tensor_by_name(prefix + 'rate/Estimated_Bpp:0')
    psnr = graph.get_tensor_by_name(prefix + 'distortion/PSNR:0')
    # reconstructed frame
    reconframe = graph.get_tensor_by_name(prefix + 'ReconFrame:0')

    with tf.Session(graph=graph) as sess:

        im1 = imread(input_path)
        im2 = imread(refer_path)
        im1 = im1 / 255.0
        im2 = im2 / 255.0
        im1 = np.expand_dims(im1, axis=0)
        im2 = np.expand_dims(im2, axis=0)

        bpp_est, Res_q, Res_prior_q, motion_q, psnr_val, recon_val = sess.run(
            [bpp, Res, Res_prior, motion, psnr, reconframe], feed_dict={
                inputImage: im1,
                previousImage: im2
            })

    print(bpp_est)
    print(psnr_val)
    if not os.path.exists(outputfolder):
        os.mkdir(outputfolder)

    output = open(outputfolder + 'quantized_res_feature.pkl', 'wb')
    pickle.dump(Res_q, output)

    output = open(outputfolder + 'quantized_res_prior_feature.pkl', 'wb')
    pickle.dump(Res_prior_q, output)

    output = open(outputfolder + 'quantized_motion_feature.pkl', 'wb')
    pickle.dump(motion_q, output)




In [ ]:
EncoderModel='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/model/L2048/frozen_model_E.pb'
input_frame= '/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im002.png'
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im001.png'
outputpath= './outputfolder/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
encoder(EncoderModel,input_frame,refer_frame,outputpath)

0.022009527
67.603516


In [ ]:

def CalcuPSNR(target, ref):
    diff = ref - target
    diff = diff.flatten('C')
    rmse = math.sqrt(np.mean(diff**2.))
    return 20 * math.log10(1.0 / (rmse))


def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph


def decoder(loadmodel, refer_path, outputfolder,imagedir):
    graph = load_graph(loadmodel)

    reconframe = graph.get_tensor_by_name('import/build_towers/tower_0/train_net_inference_one_pass/train_net/ReconFrame:0')
    res_input = graph.get_tensor_by_name('import/quant_feature:0')
    res_prior_input = graph.get_tensor_by_name('import/quant_z:0')
    motion_input = graph.get_tensor_by_name('import/quant_mv:0')
    previousImage = graph.get_tensor_by_name('import/input_image_ref:0')

    with tf.Session(graph=graph) as sess:

        with open(outputfolder + 'quantized_res_feature.pkl', 'rb') as f:
            residual_feature = pickle.load(f)

        with open(outputfolder + 'quantized_res_prior_feature.pkl', 'rb') as f:
            residual_prior_feature = pickle.load(f)

        with open(outputfolder + 'quantized_motion_feature.pkl', 'rb') as f:
            motion_feature = pickle.load(f)

        im1 = imread(refer_path)
        im1 = im1 / 255.0
        im1 = np.expand_dims(im1, axis=0)

        # reconstructed image
        recon_d = sess.run(
            [reconframe],
            feed_dict={
                res_input: residual_feature,
                res_prior_input: residual_prior_feature,
                motion_input: motion_feature,
                previousImage: im1
            })
        
        
        
        #check 
        # imagedir = '/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/'
        im2 = imread(imagedir)
        im2 = im2 / 255.0
        im2 = np.expand_dims(im2, axis=0)
        print(CalcuPSNR(im2, recon_d))

        return recon_d,CalcuPSNR(im2, recon_d)

In [ ]:
DecoderModel='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/model/L2048/frozen_model_D.pb'
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im001.png'
loadpath='./outputfolder/'

In [15]:
r,psnr=decoder(DecoderModel,refer_frame,loadpath,refer_frame)

67.70421888270045


In [ ]:
r=np.array(r)
r=r.reshape(448, 832, 3)

r.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

min_val = np.min(r)
max_val = np.max(r)

# Scale the pixel values to the range [0, 1]
r_norm =( (r - min_val) / (max_val - min_val))

# Display the normalized image
plt.imshow(r_norm)

plt.show()


In [ ]:
plt.imshow(plt.imread('/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im002.png'))
plt.show()

Encoding Decoding images


In [ ]:
input_frame= '/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im005.png'
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im004.png'

In [ ]:
encoder(EncoderModel,input_frame,refer_frame,outputpath)

In [ ]:
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im004.png'
loadpath='./outputfolder/'

In [ ]:
r,psnr=decoder(DecoderModel,refer_frame,loadpath,refer_frame)

print(psnr)

r=np.array(r)
print(r.shape)
r=r.reshape(1024, 1920, 3)

min_val = np.min(r)
max_val = np.max(r)

# Scale the pixel values to the range [0, 1]
r_norm =( (r - min_val) / (max_val - min_val))

# Display the normalized image
plt.imshow(r_norm)

plt.show()

In [ ]:
plt.imshow(plt.imread('/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im005.png'))
plt.show()

In [ ]:
plt.imshow(plt.imread('/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im004.png'))
plt.show()

Image 2

In [ ]:
input_frame= '/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im022.png'
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im021.png'

In [ ]:
encoder(EncoderModel,input_frame,refer_frame,outputpath)

In [ ]:
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im021.png'
loadpath='./outputfolder/'

In [ ]:
r,psnr=decoder(DecoderModel,refer_frame,loadpath,refer_frame)

print(psnr)

r=np.array(r)
print(r.shape)
r=r.reshape(1024, 1920, 3)

min_val = np.min(r)
max_val = np.max(r)

# Scale the pixel values to the range [0, 1]
r_norm =( (r - min_val) / (max_val - min_val))

# Display the normalized image
plt.imshow(r_norm)

plt.show()

In [ ]:
plt.imshow(plt.imread('/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im022.png'))
plt.show()

In [ ]:
plt.imshow(plt.imread('/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im021.png'))
plt.show()

Image 3

In [ ]:
input_frame= '/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im007.png'
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im006.png'

In [ ]:
encoder(EncoderModel,input_frame,refer_frame,outputpath)

In [ ]:
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im006.png'
loadpath='./outputfolder/'

In [ ]:
r,psnr=decoder(DecoderModel,refer_frame,loadpath,refer_frame)

print(psnr)

r=np.array(r)
print(r.shape)
r=r.reshape(1024, 1920, 3)

min_val = np.min(r)
max_val = np.max(r)

# Scale the pixel values to the range [0, 1]
r_norm =( (r - min_val) / (max_val - min_val))

# Display the normalized image
plt.imshow(r_norm)

plt.show()

In [ ]:
imgg=plt.imread('/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im007.png')
plt.imshow(imgg)
plt.show()

In [ ]:
plt.imshow(plt.imread('/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im006.png'))
plt.show()

Image 4

In [ ]:
input_frame= '/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im009.png'
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im008.png'

In [ ]:
encoder(EncoderModel,input_frame,refer_frame,outputpath)

In [ ]:
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im008.png'
loadpath='./outputfolder/'

In [ ]:
r,psnr=decoder(DecoderModel,refer_frame,loadpath,refer_frame)

print(psnr)

r=np.array(r)
print(r.shape)
r=r.reshape(1024, 1920, 3)

min_val = np.min(r)
max_val = np.max(r)

# Scale the pixel values to the range [0, 1]
r_norm =( (r - min_val) / (max_val - min_val))

# Display the normalized image
plt.imshow(r_norm)

plt.show()

In [ ]:
plt.imshow(plt.imread('/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im009.png'))
plt.show()

In [ ]:
plt.imshow(plt.imread('/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im008.png'))
plt.show()

Image 5

In [ ]:
input_frame= '/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im011.png'
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im010.png'

In [ ]:
encoder(EncoderModel,input_frame,refer_frame,outputpath)

In [ ]:
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im010.png'
loadpath='./outputfolder/'

In [ ]:
r,psnr=decoder(DecoderModel,refer_frame,loadpath,refer_frame)

print(psnr)

r=np.array(r)
print(r.shape)
r=r.reshape(1024, 1920, 3)

min_val = np.min(r)
max_val = np.max(r)

# Scale the pixel values to the range [0, 1]
r_norm =( (r - min_val) / (max_val - min_val))

# Display the normalized image
plt.imshow(r_norm)

plt.show()

In [ ]:
plt.imshow(plt.imread('/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im011.png'))
plt.show()

In [ ]:
plt.imshow(plt.imread('/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im010.png'))
plt.show()

Image 6

In [ ]:
input_frame= '/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im013.png'
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im012.png'

In [ ]:
encoder(EncoderModel,input_frame,refer_frame,outputpath)

In [ ]:
refer_frame='/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im012.png'
loadpath='./outputfolder/'

In [ ]:
r,psnr=decoder(DecoderModel,refer_frame,loadpath,refer_frame)

print(psnr)

r=np.array(r)
print(r.shape)
r=r.reshape(1024, 1920, 3)

min_val = np.min(r)
max_val = np.max(r)

# Scale the pixel values to the range [0, 1]
r_norm =( (r - min_val) / (max_val - min_val))

# Display the normalized image
plt.imshow(r_norm)

plt.show()

In [ ]:
plt.imshow(plt.imread('/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im013.png'))
plt.show()

In [ ]:
plt.imshow(plt.imread('/content/drive/MyDrive/dc2/DVC-master/DVC-master/TestDemo/VideoCodec/image/im012.png'))
plt.show()